# Phase 1 - Week 2 - Day 5 AM - Technical Test: SQL

## A. Overview

This technical assessment evaluates SQL querying skills across various difficulty levels, from basic `SELECT` statements to complex analytical queries. The test consists of 6 questions that progressively increase in complexity, allowing candidates to demonstrate their proficiency in SQL.

### A.1 - Skills Evaluated

* Basic querying and filtering (`SELECT`, `WHERE`)
* Table relationships and `JOIN`s
* Aggregation and grouping (`GROUP BY`, `HAVING`)
* Subqueries and Common Table Expressions (CTEs)
* Window functions
* Advanced analytical queries

### A.2 - Dataset Description

The assessment uses the Chinook Database, which models a digital media store similar to iTunes. The database follows a normalized structure with proper foreign key relationships. You could download the dataset on this [link](https://github.com/FTDS-learning-materials/phase-1/raw/refs/heads/master/w2/P1W2D5AM%20-%20Technical%20Test%202%20-%20SQL%20-%20Export%20Files.sql).

**Key Tables and Their Structures**

*Related to Core Music Data*
1.  **Album**
  * album_id (PK)
  * title
  * artist_id (FK)

2. **Artist**
  * artist_id (PK)
  * name

3. **Track**
  * track_id (PK)
  * name
  * album_id (FK)
  * media_type_id (FK)
  * genre_id (FK)
  * composer
  * milliseconds
  * bytes
  * unit_price

4. **Genre**
  * genre_id (PK)
  * name

*Related to Sales and Customer Data*
1. **Customer**
  * customer_id (PK)
  * first_name
  * last_name
  * email
  * other customer details

2. **Invoice**
  * invoice_id (PK)
  * customer_id (FK)
  * invoice_date
  * total

3. **InvoiceLine**
  * invoice_line_id (PK)
  * invoice_id (FK)
  * track_id (FK)
  * unit_price
  * quantity

*Related to Playlist Management*
1. **Playlist**
  * playlist_id (PK)
  * name

2. **PlaylistTrack**
  * playlist_id (FK)
  * track_id (FK)

### A.3 - Important Notes

1. All monetary values are in USD
2. Track lengths are stored in milliseconds
3. Column names use snake_case convention
4. All primary and foreign key relationships are properly indexed

### A.4 - Tips for Success

1. Read each question carefully before starting
2. Consider edge cases in your solutions
3. Test your queries with different scenarios
4. Focus on both functionality and performance
5. Use appropriate table aliases for better readability
6. Include necessary comments to explain complex logic

### A.5 - Database Properties

1. Fully normalized database structure
2. Referential integrity enforced through foreign keys
3. Indexes on frequently queried columns
4. Contains sample data across multiple years
5. Realistic business scenarios and relationships


Remember that the questions are designed to test both your knowledge of SQL fundamentals and your ability to solve complex business problems using SQL.

## B. Questions

### B.1 - Find all tracks that have a length greater than 5 minutes (length is stored in milliseconds)

```sql
SELECT Name, Milliseconds/1000 as Seconds
FROM Track
WHERE Milliseconds > 300000
ORDER BY Milliseconds DESC;
```

### B.2 - List all albums and their artists, showing only albums from the 'Rock' genre


```sql
SELECT DISTINCT ar.Name as ArtistName,
       al.Title as AlbumTitle
FROM Artist ar
JOIN Album al ON ar.Artist_Id = al.Artist_Id
JOIN Track t ON al.Album_Id = t.Album_Id
JOIN Genre g ON t.Genre_Id = g.Genre_Id
WHERE g.Name = 'Rock'
ORDER BY ar.Name;
```

### B.3 - Find genres that have more than 100 tracks and show their average track length

```sql
SELECT g.Name as Genre,
       COUNT(*) as TrackCount,
       ROUND(AVG(t.Milliseconds/1000.0), 2) as AvgLengthInSeconds
FROM Genre g
JOIN Track t ON g.Genre_Id = t.Genre_Id
GROUP BY g.Name
HAVING COUNT(*) > 100
ORDER BY TrackCount DESC;
```

### B.4 - Find the top 3 artists by number of tracks in each genre

```sql
WITH track_counts AS (
    SELECT ar.artist_id,
           ar.name as artist_name,
           g.name as genre_name,
           COUNT(*) as track_count,
           ROW_NUMBER() OVER (PARTITION BY g.name ORDER BY COUNT(*) DESC) as rank_in_genre
    FROM Artist ar
    JOIN Album al ON ar.artist_id = al.artist_id
    JOIN Track t ON al.album_id = t.album_id
    JOIN Genre g ON t.genre_id = g.genre_id
    GROUP BY ar.artist_id, ar.name, g.name
)
SELECT artist_name,
       genre_name,
       track_count
FROM track_counts
WHERE rank_in_genre <= 3
ORDER BY genre_name, rank_in_genre;
```

## C. Free Practice

### C.1 - Find all customers who have spent more than the average amount spent by all customers

```sql
SELECT c.First_Name,
       c.Last_Name,
       ROUND(SUM(i.Total), 2) as TotalSpent
FROM Customer c
JOIN Invoice i ON c.Customer_Id = i.Customer_Id
GROUP BY c.Customer_Id, c.First_Name, c.Last_Name
HAVING SUM(i.Total) > (
    SELECT AVG(CustomerTotal)
    FROM (
        SELECT SUM(Total) as CustomerTotal
        FROM Invoice
        GROUP BY Customer_Id
    ) as CustomerAverages
)
ORDER BY TotalSpent DESC;
```

### C.2 - Find customers who have purchased all songs from at least one genre

```sql
SELECT DISTINCT c.first_name,
                c.last_name,
                g.name as genre_name
FROM Customer c
JOIN Invoice i ON c.customer_id = i.customer_id
JOIN Invoice_Line il ON i.invoice_id = il.invoice_id
JOIN Track t ON il.track_id = t.track_id
JOIN Genre g ON t.genre_id = g.genre_id
WHERE NOT EXISTS (
    SELECT t2.track_id
    FROM Track t2
    WHERE t2.genre_id = g.genre_id
    AND NOT EXISTS (
        SELECT 1
        FROM Invoice i2
        JOIN Invoice_Line il2 ON i2.invoice_id = il2.invoice_id
        WHERE i2.customer_id = c.customer_id
        AND il2.track_id = t2.track_id
    )
)
ORDER BY g.name, c.last_name;
```